In [1]:
import numpy as np

# Beispielwerte für die Winkel (in Grad) und Beschleunigungen (in g)
pitch_deg = 30   # Neigung um die X-Achse
roll_deg = 20    # Neigung um die Y-Achse
# Angenommene Beschleunigung, wenn der Sensor perfekt aufrecht steht
# Diese Werte sollten die Gravitationskraft reflektieren
acc_x_g = 0
acc_y_g = 0
acc_z_g = 1

# Konvertiere Winkel von Grad in Bogenmaß
pitch = np.radians(pitch_deg)
roll = np.radians(roll_deg)

# Konvertiere Beschleunigungen von g in m/s²
acc_x = acc_x_g * 9.81
acc_y = acc_y_g * 9.81
acc_z = acc_z_g * 9.81

# Berechne die Richtung des Beschleunigungsvektors
# Korrigiere die Beschleunigungswerte für die Neigung des Sensors
acc_x_corrected = acc_x * np.cos(pitch) - acc_z * np.sin(pitch)
acc_y_corrected = acc_x * np.sin(roll) * np.sin(pitch) + acc_y * np.cos(roll) - acc_z * np.sin(roll) * np.cos(pitch)
acc_z_corrected = acc_x * np.cos(roll) * np.sin(pitch) + acc_y * np.sin(roll) + acc_z * np.cos(pitch) * np.cos(roll)

# Berechne die Summe der Gravitationskomponenten
gravity_magnitude = np.sqrt(acc_x_corrected**2 + acc_y_corrected**2 + acc_z_corrected**2)

# Ausgabe der korrigierten Beschleunigungswerte und der Summe der Gravitationskomponenten
print(f"Beschleunigung nach Korrektur (m/s²):")
print(f"X: {acc_x_corrected:.2f}")
print(f"Y: {acc_y_corrected:.2f}")
print(f"Z: {acc_z_corrected:.2f}")

print(f"\nGesamte Gravitationskomponente (m/s²): {gravity_magnitude:.2f}")


Beschleunigung nach Korrektur (m/s²):
X: -4.90
Y: -2.91
Z: 7.98

Gesamte Gravitationskomponente (m/s²): 9.81


In [7]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Beispielwerte für die Winkel (in Grad) und Beschleunigungen (in g)
angles = {
    'Pitch': np.linspace(0, 90, 10),  # 10 Werte von 0 bis 90 Grad für Pitch
    'Roll': np.linspace(0, 90, 10),   # 10 Werte von 0 bis 90 Grad für Roll
    'Yaw': np.linspace(0, 360, 10)    # 10 Werte von 0 bis 360 Grad für Yaw
}
acc_x_g = 0
acc_y_g = 0
acc_z_g = 1

# Erstelle eine leere Liste für die Daten
data = []

# Schleife über alle Kombinationen von Pitch, Roll und Yaw
for pitch_deg in angles['Pitch']:
    for roll_deg in angles['Roll']:
        for yaw_deg in angles['Yaw']:
            # Konvertiere Winkel von Grad in Bogenmaß
            pitch = np.radians(pitch_deg)
            roll = np.radians(roll_deg)
            yaw = np.radians(yaw_deg)

            # Konvertiere Beschleunigungen von g in m/s²
            acc_x = acc_x_g * 9.81
            acc_y = acc_y_g * 9.81
            acc_z = acc_z_g * 9.81

            # Berechne die Rotationsmatrizen
            R_x = np.array([
                [1, 0, 0],
                [0, np.cos(pitch), -np.sin(pitch)],
                [0, np.sin(pitch), np.cos(pitch)]
            ])

            R_y = np.array([
                [np.cos(roll), 0, np.sin(roll)],
                [0, 1, 0],
                [-np.sin(roll), 0, np.cos(roll)]
            ])

            R_z = np.array([
                [np.cos(yaw), -np.sin(yaw), 0],
                [np.sin(yaw), np.cos(yaw), 0],
                [0, 0, 1]
            ])

            # Gesamte Rotationsmatrix
            R = R_z @ R_y @ R_x

            # Originaler Beschleunigungsvektor
            acc = np.array([acc_x, acc_y, acc_z])

            # Berechne den korrigierten Beschleunigungsvektor
            acc_corrected = R @ acc

            # Berechne die Summe der Gravitationskomponenten
            gravity_magnitude = np.sqrt(acc_corrected[0]**2 + acc_corrected[1]**2 + acc_corrected[2]**2)

            # Speichern der Ergebnisse in der Liste
            data.append({
                'Pitch': pitch_deg,
                'Roll': roll_deg,
                'Yaw': yaw_deg,
                'Acc_X': acc_corrected[0],
                'Acc_Y': acc_corrected[1],
                'Acc_Z': acc_corrected[2],
                'Gravity_Magnitude': gravity_magnitude
            })

# Erstelle einen DataFrame aus den gesammelten Daten
df = pd.DataFrame(data)

# Füge eine Spalte mit fortlaufenden Nummern hinzu (beginnend bei 1)
df.insert(0, 'ID', range(1, len(df) + 1))

# Alle numerischen Spalten in Strings umwandeln und den Punkt durch Komma ersetzen
df = df.applymap(lambda x: str(x).replace('.', ',') if isinstance(x, float) else x)

# Speichern des DataFrames als CSV-Datei mit Komma als Trennzeichen
csv_filename = 'imu_data_comma_with_id.csv'
df.to_csv(csv_filename, index=False, sep=',')

print(f'Daten erfolgreich in {csv_filename} gespeichert.')

# Erstelle den Plotly 3D Scatter Plot
fig = go.Figure(data=[go.Scatter3d(
    x=df['Pitch'],
    y=df['Roll'],
    z=df['Yaw'],
    mode='markers',
    marker=dict(
        size=5,
        color=df['Gravity_Magnitude'].astype(float),  # Farbskala nach Gravitationskomponente
        colorscale='Viridis',
        colorbar=dict(title='Gravitationskomponente (m/s²)')
    ),
    text=df[['ID', 'Pitch', 'Roll', 'Yaw', 'Gravity_Magnitude']],
    hoverinfo='text'
)])

# Layout für die 3D-Darstellung
fig.update_layout(
    scene=dict(
        xaxis_title='Pitch (°)',
        yaxis_title='Roll (°)',
        zaxis_title='Yaw (°)',
    ),
    title='Gravitationskomponente vs. Pitch, Roll und Yaw'
)

# Zeige die interaktive Grafik
fig.show()


Daten erfolgreich in imu_data_comma_with_id.csv gespeichert.


C:\Users\Prozesstechnik\AppData\Local\Temp\ipykernel_26040\1289477576.py:81: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



ValueError: could not convert string to float: '9,81'